In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sentence_transformers import SentenceTransformer
import faiss

In [3]:
df = pd.read_csv("../data/processed/filtered_complaints.csv")

# Use the correct column name
df = df.dropna(subset=["cleaned_narrative"]).reset_index(drop=True)

df.shape


(80667, 21)

In [9]:
# Drop rows where the cleaned narrative is empty
# Adjust column name according to your preprocessing script: 'cleaned_narrative'
df = df.dropna(subset=['cleaned_narrative']).reset_index(drop=True)

print(f"✅ Dataset shape after removing empty narratives: {df.shape}")


✅ Dataset shape after removing empty narratives: (80667, 21)


In [11]:
from sklearn.model_selection import train_test_split

# Optional: limit dataset size with stratified sampling
sample_size = 15000  # or 10000
df_sampled, _ = train_test_split(
    df,
    train_size=sample_size,
    stratify=df["Product"],  # ensures balanced distribution across products
    random_state=42
)
df = df_sampled.reset_index(drop=True)

In [12]:
def chunk_text(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size - overlap):
        chunk = words[i:i + chunk_size]
        chunks.append(" ".join(chunk))

    return chunks